In [26]:
from Utils import load_pickle, save_pickle


high_50 = load_pickle('high_50','/workspace/data/output/set50/high_noisy_50/results/array_dual_nlm_high_50_filtereds.pkl')


In [28]:
for high in high_50:
    print(f'psnr: {high["psnr"]} | ssim: {high["ssim"]} | score: {high["score"]} | image: {high["file_name"]}')

psnr: 15.529034155647384 | ssim: 0.2685358944876725 | score: 21.191311802207316 | image: 36.gif
psnr: 15.029082468784951 | ssim: 0.1117720633242466 | score: 13.103144400604805 | image: 39.gif
psnr: 14.955490055729761 | ssim: 0.14754039906673894 | score: 14.854764981201829 | image: 4.gif
psnr: 15.367631477313221 | ssim: 0.16146864043921047 | score: 15.757247760617133 | image: 3.gif
psnr: 15.589865747053294 | ssim: 0.2788715154037486 | score: 21.738508643714077 | image: 32.gif
psnr: 15.893595716684803 | ssim: 0.37885573439319614 | score: 26.889584578002207 | image: 7.gif
psnr: 14.600522420116837 | ssim: 0.09712771868338697 | score: 12.156647144227767 | image: 40.gif
psnr: 15.151634735745436 | ssim: 0.27229945444994186 | score: 21.19079009036981 | image: 15.gif
psnr: 15.012342094991364 | ssim: 0.1839380175688543 | score: 16.703071925938396 | image: 48.gif
psnr: 14.834578559010033 | ssim: 0.15138183592448468 | score: 14.98638107572925 | image: 38.gif
psnr: 15.517656880520217 | ssim: 0.2168

In [29]:
from Utils import load_pickle, save_pickle


high_25 = load_pickle('high_25','/workspace/data/output/set50/high_noisy_25/results/array_dual_nlm_high_25_filtereds.pkl')

In [30]:
for high in high_25:
    print(f'psnr: {high["psnr"]} | ssim: {high["ssim"]} | score: {high["score"]} | image: {high["file_name"]}')

psnr: 21.30173443098871 | ssim: 0.5249121497505309 | score: 36.8964747030209 | image: 36.gif
psnr: 20.65491339813245 | ssim: 0.2694881631594895 | score: 23.801864857040698 | image: 39.gif
psnr: 20.63725752637042 | ssim: 0.3508107174423833 | score: 27.859164635304374 | image: 4.gif
psnr: 20.98351960321098 | ssim: 0.3393754165730416 | score: 27.46053063025757 | image: 3.gif
psnr: 21.739371915585195 | ssim: 0.5138733518139496 | score: 36.56335354849007 | image: 32.gif
psnr: 21.769433908998806 | ssim: 0.591998017302256 | score: 40.484617819612204 | image: 7.gif
psnr: 20.50159562745238 | ssim: 0.2667347690505848 | score: 23.58753626625543 | image: 40.gif
psnr: 20.74016069993863 | ssim: 0.5009240142529424 | score: 35.416281062616434 | image: 15.gif
psnr: 20.94061436571481 | ssim: 0.37315744681191554 | score: 29.128179523453184 | image: 48.gif
psnr: 20.625973587328197 | ssim: 0.350236850149433 | score: 27.82482930113575 | image: 38.gif
psnr: 21.499479411900225 | ssim: 0.4427492316880403 | sco

In [ ]:
import numpy as np
import skimage
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize

dir_images_general = '/workspace/data/input/general_images'
dir_DUALNLM= '/workspace/data/output/set50/high_noisy/DUALNLM'


img_3 = skimage.io.imread(f'{dir_images_general}/3.gif')
img_4 = skimage.io.imread(f'{dir_images_general}/4.gif')
img_7 = skimage.io.imread(f'{dir_images_general}/7.gif')
img_32 = skimage.io.imread(f'{dir_images_general}/32.gif')
img_36 = skimage.io.imread(f'{dir_images_general}/36.gif')
img_39 = skimage.io.imread(f'{dir_images_general}/39.gif')
img_40 = skimage.io.imread(f'{dir_images_general}/40.gif')


img_3_noise = skimage.io.imread(f'{dir_DUALNLM}/3.gif')
img_4_noise = skimage.io.imread(f'{dir_DUALNLM}/4.gif')    
img_7_noise = skimage.io.imread(f'{dir_DUALNLM}/7.gif')
img_32_noise = skimage.io.imread(f'{dir_DUALNLM}/32.gif')
img_36_noise = skimage.io.imread(f'{dir_DUALNLM}/36.gif')
img_39_noise = skimage.io.imread(f'{dir_DUALNLM}/39.gif')
img_40_noise = skimage.io.imread(f'{dir_DUALNLM}/40.gif')



arrays = [img_3, img_4, img_7, img_32, img_36, img_39, img_40]
noisy_arrays = [img_3_noise, img_4_noise, img_7_noise, img_32_noise, img_36_noise, img_39_noise, img_40_noise]

gifs = ['3.gif', '4.gif', '7.gif', '32.gif', '36.gif', '39.gif', '40.gif']

for i in range(len(arrays)):
    img = arrays[i]
   
    noisy_img = noisy_arrays[i]
    # Read image from disk

    # If the image has 4 dimensions (e.g. multi-page TIFF), use only the first slice
    if img.ndim == 4 or noisy_img.ndim == 4:
        img = img[0]
        noisy_img = noisy_img[0]

    # If the image has an alpha channel (RGBA), discard the alpha and keep RGB
    if img.ndim == 3 and img.shape[-1] == 4 or noisy_img.ndim == 3 and noisy_img.shape[-1] == 4:
        img = img[..., :3]
        noisy_img = noisy_img[..., :3]


    # Ensure the image is in [0, 255] as float32
    if img.dtype.kind == 'f' or noisy_img.dtype.kind == 'f':
        # If already float, clip to [0,1] then scale to [0,255]
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
        noisy_img = (np.clip(noisy_img, 0, 1) * 255).astype(np.float32)
    else:
        # If integer, just cast to float32
        img = img.astype(np.float32)
        noisy_img = noisy_img.astype(np.float32)

    if noisy_img.ndim == 3:
       noisy_img = noisy_img.mean(axis=2)
    

    psnr = peak_signal_noise_ratio(img, noisy_img, data_range=255)
    ssim = structural_similarity(img, noisy_img, data_range=255)

   
    # Mixed score (PSNR + scaled SSIM)
    score = 0.5 * psnr + (1 - 0.5) * (ssim * 100)
    print(f"PSNR = {psnr:.8f} | SSIM = {ssim:.8f} | Score = {score:.8f} | Image: {gifs[i]}")

PSNR = 26.22819190 | SSIM = 0.57022952 | Score = 41.62557176 | Image: 3.gif
PSNR = 25.50309374 | SSIM = 0.57456999 | Score = 41.48004648 | Image: 4.gif
PSNR = 26.18791823 | SSIM = 0.75403391 | Score = 50.79565485 | Image: 7.gif
PSNR = 26.58650848 | SSIM = 0.69489650 | Score = 48.03807916 | Image: 32.gif
PSNR = 26.31254692 | SSIM = 0.73612842 | Score = 49.96269431 | Image: 36.gif
PSNR = 26.05434821 | SSIM = 0.49888160 | Score = 37.97125422 | Image: 39.gif
PSNR = 25.48621569 | SSIM = 0.49320228 | Score = 37.40322200 | Image: 40.gif


In [ ]:
import numpy as np
import skimage
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize

dir_images_general = '/workspace/data/input/general_images'
dir_DUALNLM= '/workspace/data/output/set50/moderate_noisy/DUALNLM'


img_3 = skimage.io.imread(f'{dir_images_general}/3.gif')
img_4 = skimage.io.imread(f'{dir_images_general}/4.gif')
img_7 = skimage.io.imread(f'{dir_images_general}/7.gif')
img_32 = skimage.io.imread(f'{dir_images_general}/32.gif')
img_36 = skimage.io.imread(f'{dir_images_general}/36.gif')
img_39 = skimage.io.imread(f'{dir_images_general}/39.gif')
img_40 = skimage.io.imread(f'{dir_images_general}/40.gif')


img_3_noise = skimage.io.imread(f'{dir_DUALNLM}/3.gif')
img_4_noise = skimage.io.imread(f'{dir_DUALNLM}/4.gif')    
img_7_noise = skimage.io.imread(f'{dir_DUALNLM}/7.gif')
img_32_noise = skimage.io.imread(f'{dir_DUALNLM}/32.gif')
img_36_noise = skimage.io.imread(f'{dir_DUALNLM}/36.gif')
img_39_noise = skimage.io.imread(f'{dir_DUALNLM}/39.gif')
img_40_noise = skimage.io.imread(f'{dir_DUALNLM}/40.gif')



arrays = [img_3, img_4, img_7, img_32, img_36, img_39, img_40]
noisy_arrays = [img_3_noise, img_4_noise, img_7_noise, img_32_noise, img_36_noise, img_39_noise, img_40_noise]

gifs = ['3.gif', '4.gif', '7.gif', '32.gif', '36.gif', '39.gif', '40.gif']

for i in range(len(arrays)):
    img = arrays[i]
   
    noisy_img = noisy_arrays[i]
    # Read image from disk

    # If the image has 4 dimensions (e.g. multi-page TIFF), use only the first slice
    if img.ndim == 4 or noisy_img.ndim == 4:
        img = img[0]
        noisy_img = noisy_img[0]

    # If the image has an alpha channel (RGBA), discard the alpha and keep RGB
    if img.ndim == 3 and img.shape[-1] == 4 or noisy_img.ndim == 3 and noisy_img.shape[-1] == 4:
        img = img[..., :3]
        noisy_img = noisy_img[..., :3]


    # Ensure the image is in [0, 255] as float32
    if img.dtype.kind == 'f' or noisy_img.dtype.kind == 'f':
        # If already float, clip to [0,1] then scale to [0,255]
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
        noisy_img = (np.clip(noisy_img, 0, 1) * 255).astype(np.float32)
    else:
        # If integer, just cast to float32
        img = img.astype(np.float32)
        noisy_img = noisy_img.astype(np.float32)

    if noisy_img.ndim == 3:
       noisy_img = noisy_img.mean(axis=2)
    

    psnr = peak_signal_noise_ratio(img, noisy_img, data_range=255)
    ssim = structural_similarity(img, noisy_img, data_range=255)

   
    # Mixed score (PSNR + scaled SSIM)
    score = 0.5 * psnr + (1 - 0.5) * (ssim * 100)
    print(f"PSNR = {psnr:.8f} | SSIM = {ssim:.8f} | Score = {score:.8f} | Image: {gifs[i]}")

PSNR = 32.63517391 | SSIM = 0.88061413 | Score = 60.34829336 | Image: 3.gif
PSNR = 32.16089987 | SSIM = 0.85801879 | Score = 58.98138968 | Image: 4.gif
PSNR = 29.95834845 | SSIM = 0.88850467 | Score = 59.40440751 | Image: 7.gif
PSNR = 30.15550358 | SSIM = 0.80797922 | Score = 55.47671264 | Image: 32.gif
PSNR = 30.89974165 | SSIM = 0.89596764 | Score = 60.24825296 | Image: 36.gif
PSNR = 33.99790230 | SSIM = 0.90603402 | Score = 62.30065215 | Image: 39.gif
PSNR = 33.47639971 | SSIM = 0.83998475 | Score = 58.73743737 | Image: 40.gif


In [24]:
import numpy as np
import skimage
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize

dir_images_general = '/workspace/data/input/general_images'
dir_DUALNLM= '/workspace/data/output/set50/low_noisy/DUALNLM'


img_3 = skimage.io.imread(f'{dir_images_general}/3.gif')
img_4 = skimage.io.imread(f'{dir_images_general}/4.gif')
img_7 = skimage.io.imread(f'{dir_images_general}/7.gif')
img_32 = skimage.io.imread(f'{dir_images_general}/32.gif')
img_36 = skimage.io.imread(f'{dir_images_general}/36.gif')
img_39 = skimage.io.imread(f'{dir_images_general}/39.gif')
img_40 = skimage.io.imread(f'{dir_images_general}/40.gif')


img_3_noise = skimage.io.imread(f'{dir_DUALNLM}/3.gif')
img_4_noise = skimage.io.imread(f'{dir_DUALNLM}/4.gif')    
img_7_noise = skimage.io.imread(f'{dir_DUALNLM}/7.gif')
img_32_noise = skimage.io.imread(f'{dir_DUALNLM}/32.gif')
img_36_noise = skimage.io.imread(f'{dir_DUALNLM}/36.gif')
img_39_noise = skimage.io.imread(f'{dir_DUALNLM}/39.gif')
img_40_noise = skimage.io.imread(f'{dir_DUALNLM}/40.gif')



arrays = [img_3, img_4, img_7, img_32, img_36, img_39, img_40]
noisy_arrays = [img_3_noise, img_4_noise, img_7_noise, img_32_noise, img_36_noise, img_39_noise, img_40_noise]

gifs = ['3.gif', '4.gif', '7.gif', '32.gif', '36.gif', '39.gif', '40.gif']

for i in range(len(arrays)):
    img = arrays[i]
   
    noisy_img = noisy_arrays[i]
    # Read image from disk

    # If the image has 4 dimensions (e.g. multi-page TIFF), use only the first slice
    if img.ndim == 4 or noisy_img.ndim == 4:
        img = img[0]
        noisy_img = noisy_img[0]

    # If the image has an alpha channel (RGBA), discard the alpha and keep RGB
    if img.ndim == 3 and img.shape[-1] == 4 or noisy_img.ndim == 3 and noisy_img.shape[-1] == 4:
        img = img[..., :3]
        noisy_img = noisy_img[..., :3]


    # Ensure the image is in [0, 255] as float32
    if img.dtype.kind == 'f' or noisy_img.dtype.kind == 'f':
        # If already float, clip to [0,1] then scale to [0,255]
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
        noisy_img = (np.clip(noisy_img, 0, 1) * 255).astype(np.float32)
    else:
        # If integer, just cast to float32
        img = img.astype(np.float32)
        noisy_img = noisy_img.astype(np.float32)

    if noisy_img.ndim == 3:
       noisy_img = noisy_img.mean(axis=2)
    

    psnr = peak_signal_noise_ratio(img, noisy_img, data_range=255)
    ssim = structural_similarity(img, noisy_img, data_range=255)

   
    # Mixed score (PSNR + scaled SSIM)
    score = 0.5 * psnr + (1 - 0.5) * (ssim * 100)
    print(f"PSNR = {psnr:.8f} | SSIM = {ssim:.8f} | Score = {score:.8f} | Image: {gifs[i]}")
   
  

PSNR = 33.34315459 | SSIM = 0.88221572 | Score = 60.78236340 | Image: 3.gif
PSNR = 33.56391646 | SSIM = 0.85981969 | Score = 59.77294277 | Image: 4.gif
PSNR = 31.04253755 | SSIM = 0.89254693 | Score = 60.14861543 | Image: 7.gif
PSNR = 31.69594062 | SSIM = 0.85586609 | Score = 58.64127459 | Image: 32.gif
PSNR = 32.39573590 | SSIM = 0.90624935 | Score = 61.51033524 | Image: 36.gif
PSNR = 35.39388771 | SSIM = 0.91013540 | Score = 63.20371384 | Image: 39.gif
PSNR = 34.11472900 | SSIM = 0.83691934 | Score = 58.90333149 | Image: 40.gif
